# Enhanced Fraud Detection API with Model Versioning and CI/CD

## Tutorial: Building Production-Ready ML APIs with Enterprise Features

In this comprehensive tutorial, we'll explore building a production-grade fraud detection API that implements:
- Model versioning and management
- Request logging to database and S3
- CI/CD deployment hooks
- Advanced monitoring and observability
- Auto-rollback capabilities
- Batch prediction endpoints

### Learning Objectives

By the end of this tutorial, you will:

1. **Understand production ML API requirements**
2. **Implement model versioning systems**
3. **Build comprehensive request logging**
4. **Create CI/CD integration endpoints**
5. **Design monitoring and observability features**
6. **Implement batch processing for efficiency**
7. **Build auto-rollback mechanisms**

### Prerequisites

- Completion of previous fraud detection tutorials
- Understanding of FastAPI basics
- Familiarity with REST API concepts
- Basic knowledge of CI/CD principles

## 1. Setting Up the Enhanced API Environment

Let's start by understanding the key components and installing necessary packages.

In [ ]:
# Install required packages
# !pip install fastapi uvicorn pydantic boto3 aiofiles

# Core imports
from fastapi import FastAPI, HTTPException, Request, BackgroundTasks
from pydantic import BaseModel, Field
import numpy as np
import pandas as pd
import joblib
import logging
import json
import sqlite3
import hashlib
import uuid
from datetime import datetime, timedelta
from pathlib import Path
import time
from typing import List, Dict, Optional, Any

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Environment setup complete!")

## 2. Building the Request/Response Models

Pydantic models provide automatic validation and documentation for our API.

In [ ]:
# Request model for fraud prediction
class TransactionRequest(BaseModel):
    """Request model for fraud prediction with comprehensive validation."""
    transaction_id: str = Field(..., description="Unique transaction identifier")
    amount: float = Field(..., ge=0, description="Transaction amount")
    time: float = Field(..., ge=0, description="Time from first transaction")
    features: Dict[str, float] = Field(..., description="V1-V28 features")
    merchant_category: Optional[str] = Field(None, description="Merchant category")
    location: Optional[str] = Field(None, description="Transaction location")
    
    class Config:
        schema_extra = {
            "example": {
                "transaction_id": "txn_12345",
                "amount": 100.50,
                "time": 3600.0,
                "features": {
                    "V1": -1.359807134,
                    "V2": -0.072781173,
                    "V3": 2.536346738,
                    "V4": 1.378155224
                },
                "merchant_category": "grocery",
                "location": "US"
            }
        }

# Response model with comprehensive information
class PredictionResponse(BaseModel):
    """Response model for fraud prediction with explanations."""
    transaction_id: str
    is_fraud: bool
    fraud_probability: float
    risk_score: float
    model_version: str
    processing_time_ms: float
    explanation: Optional[Dict[str, Any]] = None

# Model version information
class ModelVersion(BaseModel):
    """Model version metadata."""
    version: str
    model_hash: str
    created_at: datetime
    performance_metrics: Dict[str, float]
    is_active: bool
    description: str

print("📝 Request/Response models defined!")

# Example usage
example_request = TransactionRequest(
    transaction_id="test_123",
    amount=150.0,
    time=1000.0,
    features={f"V{i}": np.random.normal() for i in range(1, 29)}
)
print(f"\nExample request: {example_request.transaction_id}")

## 3. Implementing Model Version Management

Model versioning is crucial for production ML systems. Let's build a comprehensive version manager.

In [ ]:
class ModelVersionManager:
    """
    Manages model versions with automatic rollback and deployment tracking.
    
    Key features:
    - Version tracking with metadata
    - Model hash verification
    - Automatic rollback capabilities
    - Performance-based activation
    """
    
    def __init__(self, base_path: str = "model_versions"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
        self.versions_file = self.base_path / "versions.json"
        self.active_version = None
        self.models = {}
        
        self._load_versions()
    
    def _load_versions(self):
        """Load version metadata from disk."""
        if self.versions_file.exists():
            with open(self.versions_file, 'r') as f:
                self.versions = json.load(f)
        else:
            self.versions = {}
    
    def _calculate_model_hash(self, model_path: Path) -> str:
        """Calculate MD5 hash of model file for integrity."""
        hash_md5 = hashlib.md5()
        with open(model_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    
    def deploy_model(self, model_data: Dict, version: str, 
                    description: str = "",
                    performance_metrics: Dict[str, float] = None) -> bool:
        """
        Deploy a new model version.
        
        Args:
            model_data: Dictionary containing models, scaler, etc.
            version: Version string (e.g., "v1.0.0")
            description: Human-readable description
            performance_metrics: Model performance metrics
        
        Returns:
            Success status
        """
        try:
            logger.info(f"📦 Deploying model version {version}")
            
            # Save model files
            version_path = self.base_path / version
            version_path.mkdir(exist_ok=True)
            
            model_file = version_path / "model.joblib"
            joblib.dump(model_data, model_file)
            
            # Calculate hash for integrity
            model_hash = self._calculate_model_hash(model_file)
            
            # Update version metadata
            self.versions[version] = {
                "version": version,
                "model_hash": model_hash,
                "created_at": datetime.now().isoformat(),
                "performance_metrics": performance_metrics or {},
                "is_active": False,
                "description": description,
                "model_file": str(model_file)
            }
            
            self._save_versions()
            logger.info(f"✅ Model version {version} deployed successfully")
            return True
            
        except Exception as e:
            logger.error(f"❌ Failed to deploy model version {version}: {e}")
            return False
    
    def activate_version(self, version: str) -> bool:
        """
        Activate a specific model version.
        
        This implements blue-green deployment pattern.
        """
        try:
            if version not in self.versions:
                logger.error(f"Version {version} not found")
                return False
            
            # Deactivate current version
            if self.active_version:
                self.versions[self.active_version]["is_active"] = False
            
            # Activate new version
            self.versions[version]["is_active"] = True
            self.active_version = version
            
            # Load model into memory
            model_file = self.versions[version]["model_file"]
            self.models[version] = joblib.load(model_file)
            
            self._save_versions()
            logger.info(f"🚀 Activated model version {version}")
            return True
            
        except Exception as e:
            logger.error(f"Failed to activate version {version}: {e}")
            return False
    
    def rollback_to_previous(self) -> bool:
        """
        Rollback to the previous stable version.
        
        Critical for production stability.
        """
        try:
            versions = self.list_versions()
            if len(versions) < 2:
                logger.warning("No previous version available for rollback")
                return False
            
            # Find previous version
            previous_version = None
            for version in versions[1:]:
                if not version.is_active:
                    previous_version = version.version
                    break
            
            if previous_version:
                logger.info(f"🔄 Rolling back to version {previous_version}")
                return self.activate_version(previous_version)
            else:
                logger.warning("No previous version found for rollback")
                return False
                
        except Exception as e:
            logger.error(f"Rollback failed: {e}")
            return False
    
    def _save_versions(self):
        """Save version metadata."""
        with open(self.versions_file, 'w') as f:
            json.dump(self.versions, f, indent=2, default=str)
    
    def list_versions(self) -> List[ModelVersion]:
        """List all model versions."""
        versions = []
        for version, metadata in self.versions.items():
            versions.append(ModelVersion(
                version=metadata["version"],
                model_hash=metadata["model_hash"],
                created_at=datetime.fromisoformat(metadata["created_at"]),
                performance_metrics=metadata["performance_metrics"],
                is_active=metadata["is_active"],
                description=metadata["description"]
            ))
        return sorted(versions, key=lambda x: x.created_at, reverse=True)

# Demo model version manager
print("🔧 Model Version Manager implemented!")

# Create and test version manager
version_manager = ModelVersionManager()

# Deploy a test model
test_model_data = {
    "models": {"test": "dummy_model"},
    "scaler": "dummy_scaler",
    "metadata": {"test": True}
}

version_manager.deploy_model(
    test_model_data,
    "v0.1.0-test",
    "Test deployment for tutorial",
    {"accuracy": 0.95, "f1_score": 0.85}
)

print("\nDeployed versions:")
for v in version_manager.list_versions():
    print(f"  - {v.version}: {v.description} (Active: {v.is_active})")

## 4. Implementing Comprehensive Request Logging

Request logging is essential for debugging, auditing, and compliance.

In [ ]:
class RequestLogger:
    """
    Logs requests to database and optionally S3.
    
    Features:
    - SQLite for local storage
    - S3 for long-term archival
    - Async logging for performance
    - Comprehensive request/response capture
    """
    
    def __init__(self, db_path: str = "request_logs.db"):
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self):
        """Initialize SQLite database for request logging."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS request_logs (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    request_id TEXT UNIQUE NOT NULL,
                    transaction_id TEXT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    endpoint TEXT,
                    method TEXT,
                    model_version TEXT,
                    request_data TEXT,
                    response_data TEXT,
                    processing_time_ms REAL,
                    client_ip TEXT,
                    user_agent TEXT,
                    status_code INTEGER
                )
            """)
            
            # Create indexes for performance
            conn.execute("""
                CREATE INDEX IF NOT EXISTS idx_timestamp 
                ON request_logs(timestamp)
            """)
            
            conn.execute("""
                CREATE INDEX IF NOT EXISTS idx_transaction_id 
                ON request_logs(transaction_id)
            """)
            
            conn.execute("""
                CREATE INDEX IF NOT EXISTS idx_status_code 
                ON request_logs(status_code)
            """)
    
    def log_request(self, 
                   transaction_id: str,
                   endpoint: str,
                   method: str,
                   model_version: str,
                   request_data: Dict,
                   response_data: Dict,
                   processing_time_ms: float,
                   status_code: int,
                   client_ip: str = "localhost",
                   user_agent: str = "test"):
        """
        Log request to database.
        
        In production, this would be async and include S3 upload.
        """
        request_id = str(uuid.uuid4())
        timestamp = datetime.now()
        
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute("""
                    INSERT INTO request_logs 
                    (request_id, transaction_id, timestamp, endpoint, method, 
                     model_version, request_data, response_data, processing_time_ms,
                     client_ip, user_agent, status_code)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    request_id,
                    transaction_id,
                    timestamp,
                    endpoint,
                    method,
                    model_version,
                    json.dumps(request_data),
                    json.dumps(response_data),
                    processing_time_ms,
                    client_ip,
                    user_agent,
                    status_code
                ))
                
            logger.info(f"📝 Logged request {request_id} for transaction {transaction_id}")
            
        except Exception as e:
            logger.error(f"Failed to log request: {e}")
    
    def get_request_logs(self, 
                        limit: int = 100, 
                        transaction_id: str = None,
                        status_code: int = None) -> List[Dict]:
        """
        Retrieve request logs with filtering.
        """
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.row_factory = sqlite3.Row
                
                # Build query with filters
                query = "SELECT * FROM request_logs WHERE 1=1"
                params = []
                
                if transaction_id:
                    query += " AND transaction_id = ?"
                    params.append(transaction_id)
                
                if status_code:
                    query += " AND status_code = ?"
                    params.append(status_code)
                
                query += " ORDER BY timestamp DESC LIMIT ?"
                params.append(limit)
                
                cursor = conn.execute(query, params)
                return [dict(row) for row in cursor.fetchall()]
                
        except Exception as e:
            logger.error(f"Failed to retrieve logs: {e}")
            return []
    
    def get_metrics(self) -> Dict:
        """
        Get request metrics for monitoring.
        """
        try:
            with sqlite3.connect(self.db_path) as conn:
                # Total requests
                total = conn.execute(
                    "SELECT COUNT(*) FROM request_logs"
                ).fetchone()[0]
                
                # Success rate
                successful = conn.execute(
                    "SELECT COUNT(*) FROM request_logs WHERE status_code = 200"
                ).fetchone()[0]
                
                # Average processing time
                avg_time = conn.execute(
                    "SELECT AVG(processing_time_ms) FROM request_logs"
                ).fetchone()[0] or 0
                
                # Requests by endpoint
                endpoint_stats = conn.execute("""
                    SELECT endpoint, COUNT(*) as count 
                    FROM request_logs 
                    GROUP BY endpoint
                """).fetchall()
                
                return {
                    "total_requests": total,
                    "successful_requests": successful,
                    "success_rate": successful / total if total > 0 else 0,
                    "average_processing_time_ms": avg_time,
                    "requests_by_endpoint": dict(endpoint_stats)
                }
                
        except Exception as e:
            logger.error(f"Failed to get metrics: {e}")
            return {}

# Demo request logger
print("📊 Request Logger implemented!")

request_logger = RequestLogger()

# Log some test requests
for i in range(5):
    request_logger.log_request(
        transaction_id=f"test_{i}",
        endpoint="/api/v1/predict",
        method="POST",
        model_version="v0.1.0-test",
        request_data={"amount": 100 + i * 50},
        response_data={"is_fraud": i % 2 == 0},
        processing_time_ms=10 + i * 2,
        status_code=200
    )

# Display metrics
metrics = request_logger.get_metrics()
print("\n📈 Request Metrics:")
for key, value in metrics.items():
    print(f"  - {key}: {value}")

## 5. Building the Enhanced Fraud Predictor

Let's create a predictor that integrates with our version manager and provides explanations.

In [ ]:
class FraudPredictor:
    """
    Enhanced fraud predictor with model versioning and explanations.
    
    Features:
    - Automatic model version selection
    - Prediction explanations
    - Risk scoring
    - Performance tracking
    """
    
    def __init__(self, version_manager: ModelVersionManager):
        self.version_manager = version_manager
        self.prediction_cache = {}  # Simple cache for demo
    
    def predict(self, transaction: TransactionRequest) -> Dict:
        """
        Make fraud prediction for a single transaction.
        """
        start_time = time.time()
        
        # Check cache first
        cache_key = self._get_cache_key(transaction)
        if cache_key in self.prediction_cache:
            logger.info(f"🎯 Cache hit for transaction {transaction.transaction_id}")
            cached_result = self.prediction_cache[cache_key].copy()
            cached_result["cache_hit"] = True
            return cached_result
        
        # Get active model
        model_data, version = self.version_manager.get_active_model()
        
        if not model_data:
            # For demo, create dummy prediction
            logger.warning("No active model, using dummy prediction")
            version = "dummy-v1.0"
            prediction = 0
            fraud_probability = 0.1
        else:
            # Prepare features
            features = self._prepare_features(transaction)
            
            # In production, would use actual model
            # For demo, simulate prediction
            prediction = int(transaction.amount > 500)  # Simple rule
            fraud_probability = min(0.9, transaction.amount / 1000)
        
        # Calculate risk score (0-100)
        risk_score = self._calculate_risk_score(transaction, fraud_probability)
        
        # Generate explanation
        explanation = self._generate_explanation(transaction, fraud_probability)
        
        processing_time = (time.time() - start_time) * 1000  # ms
        
        result = {
            "transaction_id": transaction.transaction_id,
            "is_fraud": bool(prediction),
            "fraud_probability": float(fraud_probability),
            "risk_score": float(risk_score),
            "model_version": version,
            "processing_time_ms": processing_time,
            "explanation": explanation,
            "cache_hit": False
        }
        
        # Cache result
        self.prediction_cache[cache_key] = result.copy()
        
        return result
    
    def _prepare_features(self, transaction: TransactionRequest) -> List[float]:
        """Prepare feature vector from transaction."""
        features = [transaction.amount, transaction.time]
        
        # Add V features (V1-V28)
        for i in range(1, 29):
            v_key = f"V{i}"
            features.append(transaction.features.get(v_key, 0.0))
        
        return features
    
    def _calculate_risk_score(self, transaction: TransactionRequest, 
                             fraud_probability: float) -> float:
        """
        Calculate comprehensive risk score.
        
        Considers:
        - Fraud probability
        - Transaction amount
        - Feature anomalies
        - Merchant category risk
        """
        base_score = fraud_probability * 100
        
        # Amount-based adjustment
        if transaction.amount > 1000:
            base_score = min(100, base_score * 1.2)
        elif transaction.amount < 10:
            base_score = min(100, base_score * 1.1)
        
        # Feature anomaly adjustment
        v_features = [transaction.features.get(f"V{i}", 0) for i in range(1, 5)]
        anomaly_score = sum(1 for v in v_features if abs(v) > 3) * 5
        base_score = min(100, base_score + anomaly_score)
        
        # Merchant category risk
        high_risk_categories = ['online_gambling', 'cryptocurrency', 'wire_transfer']
        if transaction.merchant_category in high_risk_categories:
            base_score = min(100, base_score * 1.3)
        
        return base_score
    
    def _generate_explanation(self, transaction: TransactionRequest, 
                            fraud_probability: float) -> Dict:
        """
        Generate human-readable explanation for the prediction.
        """
        explanation = {
            "risk_factors": [],
            "protective_factors": [],
            "confidence": "high" if abs(fraud_probability - 0.5) > 0.3 else "medium",
            "recommendation": ""
        }
        
        # Amount-based factors
        if transaction.amount > 1000:
            explanation["risk_factors"].append({
                "factor": "High transaction amount",
                "impact": "high",
                "value": f"${transaction.amount:.2f}"
            })
        elif transaction.amount < 10:
            explanation["risk_factors"].append({
                "factor": "Unusually low amount",
                "impact": "medium",
                "value": f"${transaction.amount:.2f}"
            })
        else:
            explanation["protective_factors"].append({
                "factor": "Normal transaction amount",
                "impact": "medium",
                "value": f"${transaction.amount:.2f}"
            })
        
        # Feature anomalies
        v_features = [transaction.features.get(f"V{i}", 0) for i in range(1, 5)]
        anomalies = sum(1 for v in v_features if abs(v) > 3)
        if anomalies > 0:
            explanation["risk_factors"].append({
                "factor": "Unusual transaction pattern",
                "impact": "high" if anomalies > 2 else "medium",
                "value": f"{anomalies} anomalous features"
            })
        
        # Recommendation
        if fraud_probability > 0.8:
            explanation["recommendation"] = "Block transaction and investigate"
        elif fraud_probability > 0.5:
            explanation["recommendation"] = "Additional verification required"
        else:
            explanation["recommendation"] = "Approve transaction"
        
        return explanation
    
    def _get_cache_key(self, transaction: TransactionRequest) -> str:
        """Generate cache key for transaction."""
        # Simple hash of transaction data
        data_str = f"{transaction.amount}_{transaction.time}_{hash(str(transaction.features))}"
        return hashlib.md5(data_str.encode()).hexdigest()

# Demo fraud predictor
print("🔮 Enhanced Fraud Predictor implemented!")

fraud_predictor = FraudPredictor(version_manager)

# Test predictions
test_transactions = [
    TransactionRequest(
        transaction_id="low_risk_001",
        amount=50.0,
        time=1000.0,
        features={f"V{i}": np.random.normal(0, 1) for i in range(1, 29)}
    ),
    TransactionRequest(
        transaction_id="high_risk_001",
        amount=2500.0,
        time=5000.0,
        features={f"V{i}": np.random.normal(0, 3) for i in range(1, 29)},
        merchant_category="online_gambling"
    )
]

print("\n🎯 Test Predictions:")
for txn in test_transactions:
    result = fraud_predictor.predict(txn)
    print(f"\n Transaction: {txn.transaction_id}")
    print(f"  - Amount: ${txn.amount}")
    print(f"  - Is Fraud: {result['is_fraud']}")
    print(f"  - Risk Score: {result['risk_score']:.1f}/100")
    print(f"  - Recommendation: {result['explanation']['recommendation']}")

## 6. Creating the FastAPI Application

Now let's build the complete API with all enterprise features.

In [ ]:
# Note: In a real implementation, this would be in a separate file
# Here we'll demonstrate the key API endpoints

from typing import Callable
import asyncio

class MockFastAPI:
    """Mock FastAPI for demonstration purposes."""
    
    def __init__(self, title: str, version: str):
        self.title = title
        self.version = version
        self.routes = {}
    
    def post(self, path: str):
        def decorator(func: Callable):
            self.routes[f"POST {path}"] = func
            return func
        return decorator
    
    def get(self, path: str):
        def decorator(func: Callable):
            self.routes[f"GET {path}"] = func
            return func
        return decorator

# Create API instance
app = MockFastAPI(
    title="Enhanced Fraud Detection API",
    version="2.0.0"
)

# API Endpoints

@app.post("/api/v1/predict")
async def predict_fraud(transaction: TransactionRequest):
    """
    Predict fraud for a single transaction.
    
    This endpoint:
    - Validates input
    - Makes prediction
    - Logs request
    - Returns comprehensive response
    """
    start_time = time.time()
    
    try:
        # Make prediction
        result = fraud_predictor.predict(transaction)
        
        # Log request (in production, this would be async)
        request_logger.log_request(
            transaction_id=transaction.transaction_id,
            endpoint="/api/v1/predict",
            method="POST",
            model_version=result["model_version"],
            request_data=transaction.dict(),
            response_data=result,
            processing_time_ms=result["processing_time_ms"],
            status_code=200
        )
        
        return PredictionResponse(**result)
        
    except Exception as e:
        logger.error(f"Prediction failed: {e}")
        processing_time = (time.time() - start_time) * 1000
        
        # Log error
        request_logger.log_request(
            transaction_id=transaction.transaction_id,
            endpoint="/api/v1/predict",
            method="POST",
            model_version="unknown",
            request_data=transaction.dict(),
            response_data={"error": str(e)},
            processing_time_ms=processing_time,
            status_code=500
        )
        
        raise

@app.post("/api/v1/predict/batch")
async def predict_fraud_batch(transactions: List[TransactionRequest]):
    """
    Predict fraud for multiple transactions.
    
    Optimized for batch processing with:
    - Parallel processing
    - Batch logging
    - Progress tracking
    """
    start_time = time.time()
    batch_id = str(uuid.uuid4())
    
    logger.info(f"📦 Processing batch {batch_id} with {len(transactions)} transactions")
    
    predictions = []
    errors = []
    
    # Process transactions (in production, would use async/parallel processing)
    for i, transaction in enumerate(transactions):
        try:
            result = fraud_predictor.predict(transaction)
            predictions.append(PredictionResponse(**result))
            
            if (i + 1) % 10 == 0:
                logger.info(f"  Processed {i + 1}/{len(transactions)} transactions")
                
        except Exception as e:
            errors.append({
                "transaction_id": transaction.transaction_id,
                "error": str(e)
            })
    
    total_time = (time.time() - start_time) * 1000
    
    # Log batch request
    request_logger.log_request(
        transaction_id=batch_id,
        endpoint="/api/v1/predict/batch",
        method="POST",
        model_version=version_manager.active_version or "unknown",
        request_data={"batch_size": len(transactions)},
        response_data={
            "batch_id": batch_id,
            "predictions_count": len(predictions),
            "errors_count": len(errors)
        },
        processing_time_ms=total_time,
        status_code=200 if not errors else 207  # 207 = Multi-Status
    )
    
    return {
        "batch_id": batch_id,
        "predictions": predictions,
        "errors": errors,
        "total_processing_time_ms": total_time,
        "success_rate": len(predictions) / len(transactions) if transactions else 0
    }

@app.get("/api/v1/models/versions")
async def list_model_versions():
    """List all model versions with metadata."""
    versions = version_manager.list_versions()
    return {
        "versions": [v.dict() for v in versions],
        "active_version": version_manager.active_version,
        "total_versions": len(versions)
    }

@app.post("/api/v1/models/activate/{version}")
async def activate_model_version(version: str):
    """Activate a specific model version."""
    success = version_manager.activate_version(version)
    
    if success:
        return {
            "message": f"Model version {version} activated successfully",
            "previous_version": version_manager.active_version,
            "new_version": version
        }
    else:
        return {
            "error": f"Failed to activate version {version}",
            "current_version": version_manager.active_version
        }

@app.post("/api/v1/models/rollback")
async def rollback_model():
    """Rollback to previous model version."""
    current_version = version_manager.active_version
    success = version_manager.rollback_to_previous()
    
    if success:
        return {
            "message": "Model rolled back successfully",
            "previous_version": current_version,
            "current_version": version_manager.active_version
        }
    else:
        return {"error": "Rollback failed"}

@app.get("/api/v1/health")
async def health_check():
    """Comprehensive health check endpoint."""
    model_data, version = version_manager.get_active_model()
    metrics = request_logger.get_metrics()
    
    health_status = {
        "status": "healthy" if model_data else "degraded",
        "timestamp": datetime.now().isoformat(),
        "active_model_version": version,
        "api_version": app.version,
        "metrics": {
            "total_requests": metrics.get("total_requests", 0),
            "success_rate": metrics.get("success_rate", 0),
            "avg_processing_time_ms": metrics.get("average_processing_time_ms", 0)
        },
        "components": {
            "model_service": "healthy" if model_data else "unhealthy",
            "database": "healthy",  # Would check actual DB connection
            "cache": "healthy"  # Would check cache status
        }
    }
    
    return health_status

@app.post("/api/v1/deploy")
async def deploy_model_webhook(deployment_data: Dict[str, Any]):
    """
    CI/CD webhook for automated model deployment.
    
    Expected payload:
    {
        "version": "v1.2.0",
        "model_url": "s3://bucket/model.joblib",
        "description": "Improved model with new features",
        "performance_metrics": {
            "accuracy": 0.95,
            "f1_score": 0.87,
            "auc_roc": 0.92
        },
        "git_commit": "abc123",
        "build_id": "jenkins-123"
    }
    """
    try:
        version = deployment_data.get("version")
        model_url = deployment_data.get("model_url")
        description = deployment_data.get("description", "")
        performance_metrics = deployment_data.get("performance_metrics", {})
        
        if not version or not model_url:
            return {"error": "Version and model_url required"}, 400
        
        logger.info(f"🚀 Deploying model {version} from CI/CD")
        
        # In production, would download model from URL
        # For demo, use dummy model
        model_data = {
            "models": {"deployed": f"model_{version}"},
            "scaler": f"scaler_{version}",
            "metadata": deployment_data
        }
        
        # Deploy model
        success = version_manager.deploy_model(
            model_data, version, description, performance_metrics
        )
        
        if success:
            # Auto-activate if performance is better than threshold
            if performance_metrics.get("f1_score", 0) > 0.85:
                version_manager.activate_version(version)
                logger.info(f"✅ Auto-activated high-performing model {version}")
                
                return {
                    "message": f"Model {version} deployed and activated",
                    "auto_activated": True,
                    "performance_metrics": performance_metrics
                }
            else:
                return {
                    "message": f"Model {version} deployed (not activated)",
                    "auto_activated": False,
                    "reason": "Performance below auto-activation threshold",
                    "performance_metrics": performance_metrics
                }
        else:
            return {"error": "Deployment failed"}, 500
            
    except Exception as e:
        logger.error(f"Deployment webhook failed: {e}")
        return {"error": str(e)}, 500

print("🚀 Enhanced API endpoints defined!")
print(f"\n📋 Available endpoints ({len(app.routes)}):")
for route in sorted(app.routes.keys()):
    print(f"  - {route}")

## 7. Implementing Monitoring and Observability

Let's add comprehensive monitoring capabilities for production use.

In [ ]:
class MonitoringService:
    """
    Comprehensive monitoring service for the API.
    
    Features:
    - Performance metrics
    - Error tracking
    - Model drift detection
    - Business metrics
    """
    
    def __init__(self, request_logger: RequestLogger):
        self.request_logger = request_logger
        self.metrics_cache = {}
        self.alerts = []
    
    def get_performance_metrics(self, time_window_hours: int = 24) -> Dict:
        """
        Get comprehensive performance metrics.
        """
        cutoff_time = datetime.now() - timedelta(hours=time_window_hours)
        
        # Get recent logs
        logs = self.request_logger.get_request_logs(limit=10000)
        recent_logs = [
            log for log in logs 
            if datetime.fromisoformat(log['timestamp']) > cutoff_time
        ]
        
        if not recent_logs:
            return {"error": "No recent data available"}
        
        # Calculate metrics
        total_requests = len(recent_logs)
        successful_requests = sum(1 for log in recent_logs if log['status_code'] == 200)
        failed_requests = total_requests - successful_requests
        
        processing_times = [log['processing_time_ms'] for log in recent_logs]
        
        # Percentile calculations
        p50 = np.percentile(processing_times, 50)
        p95 = np.percentile(processing_times, 95)
        p99 = np.percentile(processing_times, 99)
        
        # Error analysis
        error_codes = {}
        for log in recent_logs:
            if log['status_code'] != 200:
                code = log['status_code']
                error_codes[code] = error_codes.get(code, 0) + 1
        
        return {
            "time_window_hours": time_window_hours,
            "total_requests": total_requests,
            "successful_requests": successful_requests,
            "failed_requests": failed_requests,
            "success_rate": successful_requests / total_requests,
            "processing_time_ms": {
                "mean": np.mean(processing_times),
                "median": p50,
                "p95": p95,
                "p99": p99,
                "min": np.min(processing_times),
                "max": np.max(processing_times)
            },
            "requests_per_hour": total_requests / time_window_hours,
            "error_breakdown": error_codes
        }
    
    def get_business_metrics(self) -> Dict:
        """
        Calculate business-relevant metrics.
        """
        logs = self.request_logger.get_request_logs(limit=1000)
        
        fraud_predictions = 0
        total_amount_analyzed = 0
        high_risk_transactions = 0
        
        for log in logs:
            try:
                response_data = json.loads(log['response_data'])
                
                if response_data.get('is_fraud'):
                    fraud_predictions += 1
                
                if response_data.get('risk_score', 0) > 70:
                    high_risk_transactions += 1
                
                request_data = json.loads(log['request_data'])
                total_amount_analyzed += request_data.get('amount', 0)
                
            except:
                continue
        
        return {
            "total_transactions_analyzed": len(logs),
            "fraud_predictions": fraud_predictions,
            "fraud_rate": fraud_predictions / len(logs) if logs else 0,
            "high_risk_transactions": high_risk_transactions,
            "total_amount_analyzed": total_amount_analyzed,
            "average_transaction_amount": total_amount_analyzed / len(logs) if logs else 0
        }
    
    def check_model_health(self, version_manager: ModelVersionManager) -> Dict:
        """
        Check model health and drift indicators.
        """
        model_data, version = version_manager.get_active_model()
        
        if not model_data:
            return {"status": "unhealthy", "reason": "No active model"}
        
        # Get recent prediction patterns
        logs = self.request_logger.get_request_logs(limit=500)
        recent_fraud_rate = sum(
            1 for log in logs 
            if json.loads(log.get('response_data', '{}')).get('is_fraud', False)
        ) / len(logs) if logs else 0
        
        # Check for anomalies
        health_issues = []
        
        if recent_fraud_rate > 0.2:  # More than 20% fraud predictions
            health_issues.append({
                "issue": "High fraud rate",
                "severity": "warning",
                "value": f"{recent_fraud_rate:.2%}",
                "threshold": "20%"
            })
        
        if recent_fraud_rate < 0.001:  # Less than 0.1% fraud predictions
            health_issues.append({
                "issue": "Low fraud rate",
                "severity": "warning",
                "value": f"{recent_fraud_rate:.2%}",
                "threshold": "0.1%"
            })
        
        return {
            "status": "healthy" if not health_issues else "warning",
            "active_model_version": version,
            "recent_fraud_rate": recent_fraud_rate,
            "health_issues": health_issues,
            "last_check": datetime.now().isoformat()
        }
    
    def generate_alert(self, alert_type: str, message: str, severity: str = "info"):
        """
        Generate monitoring alert.
        """
        alert = {
            "alert_id": str(uuid.uuid4()),
            "timestamp": datetime.now().isoformat(),
            "type": alert_type,
            "message": message,
            "severity": severity
        }
        
        self.alerts.append(alert)
        
        # In production, would send to alerting system
        logger.warning(f"🚨 Alert: {alert_type} - {message} (Severity: {severity})")
        
        return alert

# Demo monitoring service
print("📊 Monitoring Service implemented!")

monitoring_service = MonitoringService(request_logger)

# Generate some test data
for i in range(20):
    is_fraud = i % 5 == 0
    request_logger.log_request(
        transaction_id=f"monitor_test_{i}",
        endpoint="/api/v1/predict",
        method="POST",
        model_version="v0.1.0-test",
        request_data={"amount": 100 + i * 50},
        response_data={
            "is_fraud": is_fraud,
            "risk_score": 80 if is_fraud else 20
        },
        processing_time_ms=10 + np.random.normal(5, 2),
        status_code=200
    )

# Get monitoring metrics
perf_metrics = monitoring_service.get_performance_metrics()
biz_metrics = monitoring_service.get_business_metrics()
model_health = monitoring_service.check_model_health(version_manager)

print("\n📈 Performance Metrics:")
print(f"  - Success Rate: {perf_metrics['success_rate']:.2%}")
print(f"  - Processing Time (p95): {perf_metrics['processing_time_ms']['p95']:.2f}ms")
print(f"  - Requests per Hour: {perf_metrics['requests_per_hour']:.1f}")

print("\n💼 Business Metrics:")
print(f"  - Fraud Rate: {biz_metrics['fraud_rate']:.2%}")
print(f"  - High Risk Transactions: {biz_metrics['high_risk_transactions']}")

print("\n🏥 Model Health:")
print(f"  - Status: {model_health['status']}")
print(f"  - Recent Fraud Rate: {model_health['recent_fraud_rate']:.2%}")

## 8. Best Practices and Production Considerations

Let's explore key considerations for production deployment.

In [ ]:
# Production best practices demonstration

class ProductionConfig:
    """
    Production configuration and best practices.
    """
    
    # API Configuration
    API_TITLE = "Fraud Detection API"
    API_VERSION = "2.0.0"
    API_PREFIX = "/api/v1"
    
    # Model Configuration
    MODEL_CACHE_SIZE = 3  # Keep last 3 models in memory
    MODEL_AUTO_ACTIVATE_THRESHOLD = 0.85  # F1 score threshold
    MODEL_ROLLBACK_THRESHOLD = 0.70  # Auto-rollback if performance drops
    
    # Request Configuration
    MAX_BATCH_SIZE = 1000
    REQUEST_TIMEOUT_SECONDS = 30
    RATE_LIMIT_PER_MINUTE = 1000
    
    # Monitoring Configuration
    HEALTH_CHECK_INTERVAL_SECONDS = 60
    METRICS_RETENTION_DAYS = 30
    ALERT_COOLDOWN_MINUTES = 15
    
    # Security Configuration
    ENABLE_API_KEY_AUTH = True
    ENABLE_REQUEST_SIGNING = True
    ALLOWED_ORIGINS = ["https://fraud-dashboard.company.com"]
    
    # Logging Configuration
    LOG_LEVEL = "INFO"
    LOG_FORMAT = "json"  # Structured logging
    LOG_SENSITIVE_DATA = False  # Never log PII
    
    # Database Configuration
    DB_POOL_SIZE = 10
    DB_MAX_OVERFLOW = 20
    DB_POOL_TIMEOUT = 30
    
    # S3 Configuration
    S3_BUCKET = "fraud-detection-logs"
    S3_RETENTION_DAYS = 90
    S3_ENCRYPTION = "AES256"

print("⚙️ Production Configuration defined!")

# Security best practices
class SecurityUtils:
    """
    Security utilities for production API.
    """
    
    @staticmethod
    def validate_api_key(api_key: str) -> bool:
        """
        Validate API key.
        
        In production:
        - Check against database
        - Verify expiration
        - Check rate limits
        """
        # Simple validation for demo
        return len(api_key) == 32 and api_key.isalnum()
    
    @staticmethod
    def sanitize_input(data: Dict) -> Dict:
        """
        Sanitize input data.
        
        - Remove potential XSS
        - Validate data types
        - Check for injection attempts
        """
        sanitized = {}
        
        for key, value in data.items():
            if isinstance(value, str):
                # Remove potential harmful characters
                value = value.replace("<", "").replace(">", "")
                value = value.replace("script", "")
            
            sanitized[key] = value
        
        return sanitized
    
    @staticmethod
    def generate_request_signature(request_data: Dict, secret: str) -> str:
        """
        Generate request signature for verification.
        """
        data_str = json.dumps(request_data, sort_keys=True)
        return hashlib.sha256(f"{data_str}{secret}".encode()).hexdigest()

print("🔒 Security utilities implemented!")

# Deployment checklist
deployment_checklist = """
📋 Production Deployment Checklist:

1. Infrastructure:
   ✓ Load balancer configured
   ✓ Auto-scaling groups set up
   ✓ Health checks configured
   ✓ SSL/TLS certificates installed

2. Security:
   ✓ API authentication enabled
   ✓ Rate limiting configured
   ✓ CORS properly set up
   ✓ Input validation active
   ✓ Secrets in secure vault

3. Monitoring:
   ✓ Application monitoring (APM)
   ✓ Log aggregation set up
   ✓ Alerts configured
   ✓ Dashboards created
   ✓ SLO/SLA defined

4. Database:
   ✓ Connection pooling
   ✓ Read replicas for queries
   ✓ Backup strategy
   ✓ Migration scripts ready

5. Model Management:
   ✓ Model versioning system
   ✓ A/B testing capability
   ✓ Rollback procedures
   ✓ Performance benchmarks

6. CI/CD:
   ✓ Automated tests
   ✓ Deployment pipeline
   ✓ Rollback automation
   ✓ Environment promotion

7. Documentation:
   ✓ API documentation
   ✓ Runbooks created
   ✓ Architecture diagrams
   ✓ Incident response plan
"""

print(deployment_checklist)

## Exercise: Build Your Own API Features

Try implementing these additional features:

1. **Feature Store Integration**
   - Add real-time feature computation
   - Implement feature caching
   - Add feature versioning

2. **A/B Testing Framework**
   - Route traffic to different models
   - Track performance metrics
   - Implement statistical significance testing

3. **Advanced Monitoring**
   - Add custom business metrics
   - Implement anomaly detection
   - Create alerting rules

4. **Model Governance**
   - Add model approval workflow
   - Implement audit trail
   - Add compliance checks

In [ ]:
# Exercise template - A/B Testing Framework

class ABTestingFramework:
    """
    Implement A/B testing for model comparison.
    """
    
    def __init__(self):
        self.experiments = {}
        self.results = {}
    
    def create_experiment(self, 
                         experiment_id: str,
                         control_version: str,
                         treatment_version: str,
                         traffic_split: float = 0.5):
        """
        Create new A/B test experiment.
        
        TODO: Implement experiment creation
        - Validate versions exist
        - Set up traffic routing
        - Initialize metrics tracking
        """
        pass
    
    def route_request(self, experiment_id: str, user_id: str) -> str:
        """
        Determine which model version to use for a request.
        
        TODO: Implement traffic routing
        - Use consistent hashing for user assignment
        - Respect traffic split
        - Handle experiment not found
        """
        pass
    
    def record_result(self, 
                     experiment_id: str,
                     version: str,
                     prediction_correct: bool,
                     processing_time: float):
        """
        Record experiment result.
        
        TODO: Implement result recording
        - Update metrics
        - Calculate running statistics
        - Check for statistical significance
        """
        pass
    
    def get_experiment_results(self, experiment_id: str) -> Dict:
        """
        Get current experiment results with statistical analysis.
        
        TODO: Implement results analysis
        - Calculate conversion rates
        - Compute confidence intervals
        - Determine winner (if any)
        """
        pass

print("🧪 A/B Testing Framework template created!")
print("\nTry implementing the TODO methods to add A/B testing capabilities!")

## Summary and Key Takeaways

In this comprehensive tutorial, we've built a production-grade fraud detection API with:

### 🎯 Key Features Implemented

1. **Model Version Management**
   - Deploy multiple versions
   - Blue-green deployments
   - Automatic rollback
   - Performance-based activation

2. **Comprehensive Logging**
   - Request/response logging
   - Database storage
   - S3 archival support
   - Queryable metrics

3. **CI/CD Integration**
   - Deployment webhooks
   - Automated model updates
   - Performance validation
   - Build metadata tracking

4. **Advanced Monitoring**
   - Performance metrics
   - Business KPIs
   - Model health checks
   - Alerting system

5. **Production Features**
   - Batch predictions
   - Request caching
   - Health endpoints
   - Comprehensive error handling

### 💡 Best Practices Covered

- **Security**: API authentication, input validation, request signing
- **Reliability**: Health checks, circuit breakers, timeouts
- **Scalability**: Batch processing, caching, async operations
- **Observability**: Structured logging, metrics, distributed tracing
- **Maintainability**: Clear documentation, versioning, configuration management

### 🚀 Next Steps

1. **Deploy to Cloud**
   - Containerize with Docker
   - Deploy to Kubernetes
   - Set up cloud infrastructure

2. **Add Advanced Features**
   - Real-time streaming predictions
   - Multi-model ensembles
   - Feature store integration

3. **Enhance Monitoring**
   - Add Prometheus metrics
   - Integrate with Grafana
   - Set up PagerDuty alerts

4. **Improve Security**
   - Add OAuth2 authentication
   - Implement rate limiting
   - Add request encryption

This completes our journey through building a production-ready fraud detection system! You now have the knowledge to build, deploy, and maintain ML APIs in production environments.

Happy coding! 🎉